# Kaggle Intermediate ML XGBoost Exercise

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Read the data
X = pd.read_csv('./train.csv', index_col='Id')
X_test_full = pd.read_csv('./test.csv', index_col='Id')

# Remove rows with missing target (y), separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Train/test split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# 'Cardinality' means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [ col_name for col_name in X_train_full.columns if X_train_full[col_name].nunique() < 10 and
                       X_train_full[col_name].dtype == "object"]

# Select numeric columns
numeric_cols = [col_name for col_name in X_train_full.columns if X_train_full[col_name].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)

X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [13]:
# Build our model
my_model_1 = XGBRegressor(random_state=0)

# Fit our model
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=0, ...)

In [14]:
# Get our predictions for my_model_1
predictions_1 = my_model_1.predict(X_valid)

# Calculate MAE
mae_1 = mean_absolute_error(y_valid, predictions_1)
print('MAE 1:', mae_1)

MAE 1: 17662.736729452055


In [15]:
# Improve our model
# Use n_estimators and learning_rate to tune our XGBRegressor model

my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit model
my_model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(y_valid, predictions_2)

print('MAE 2:', mae_2)

MAE 2: 16688.691513270547


In [16]:
# Let's break our model


# Define model
my_model_3 = XGBRegressor(n_estimators=10, learning_rate=0.5)

# Fit model
my_model_3.fit(X_train, y_train)

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

mae_3 = mean_absolute_error(y_valid, predictions_3)

print('MAE 3:', mae_3)

MAE 3: 20568.527838720034
